In [5]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_17.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "17_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "17_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "17_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "17_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "17_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "17_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 17_sub_mat_1.csv has been saved.
Submatrix 17_sub_mat_2.csv has been saved.
Submatrix 17_sub_mat_3.csv has been saved.
submatrix 17_cross_matrix_1_2.csv has been saved
submatrix 17_cross_matrix_1_3.csv has been saved
submatrix 17_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,0.379409,0.306363,0.101226,0.428756,0.726935,0.373236,0.615019,0.113202,0.606912,0.365455,...,0.379405,0.567962,0.544354,0.49752,0.580006,0.709698,0.826367,0.701693,0.477483,0.288282
21085.0,0.434013,0.506314,0.393689,0.550842,0.661439,0.465534,0.575925,0.288898,0.575942,0.519877,...,0.488847,0.737175,0.559779,0.530027,0.672595,0.791232,0.783488,0.683722,0.639332,0.423846
21087.0,0.408397,0.361223,0.196511,0.490046,-0.48198,0.403873,0.455249,0.315159,0.473081,0.48097,...,0.388309,0.374654,0.325103,0.333518,0.433525,-0.359507,-0.387502,0.486682,0.505427,0.195707
21089.0,-0.409805,0.477417,0.393708,0.505327,0.613912,0.424716,0.565985,0.260622,0.549476,0.478114,...,0.369173,0.749512,0.638625,0.573477,0.723236,0.860985,0.838867,0.665792,0.637243,-0.412815
21091.0,-0.350901,0.341781,0.21109,0.474437,0.639785,0.352469,0.493535,0.112172,0.497954,0.411853,...,0.370125,0.668067,0.517497,0.568537,0.632413,0.717608,0.727038,0.668833,0.491267,0.390103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.558335,0.870129,0.73579,0.813444,-0.673081,-0.733582,-0.730549,0.718666,-0.590512,-0.76794,...,-0.540042,-0.731304,-0.433604,-0.65939,-0.739221,-0.520027,-0.399287,-0.599184,-0.810685,0.529007
38091.0,-0.437012,0.625434,0.545746,0.497378,-0.475791,-0.545686,-0.564165,0.498091,-0.563564,0.54153,...,-0.397528,-0.507616,0.300289,-0.38307,-0.559649,-0.487115,-0.283523,-0.409555,0.649616,0.39233
38093.0,0.615216,0.761027,0.589207,0.740422,-0.713642,0.740455,-0.669357,0.641615,-0.637866,0.720633,...,-0.558501,-0.639875,-0.393425,-0.59113,-0.599105,-0.517677,-0.430371,-0.595192,0.714946,-0.586228
38095.0,0.393565,0.527256,0.487458,0.406872,-0.124091,0.374408,-0.334884,0.508978,0.182442,-0.403013,...,0.346037,-0.387734,-0.317589,-0.478555,-0.301947,0.284794,0.243086,-0.264148,-0.490008,-0.589551
